# Testing the Named Entities Recognition engine of Spacy

### Step 1. Loading Spacy models

To free-up some memory, we do not load pipelines which we are not going to use, i.e. the tokenizer, the parser and the lemmatizer. We install the language library for the first run. Then we can comment-out the _download_ command.

In [1]:
import re
import pandas as pd
import spacy
import sys
from collections import Counter
import re

## Uncomment to install the language library
!{sys.executable} -m spacy download en

nlp = spacy.load('en_core_web_sm',disable=['tokenizer','parser','lemmatizer'])
print('Finished loading.')

Finished loading.


### Step 2. Pre-processing

* Read two files exported from the database which contain the SE Glossary articles definitions and their titles.
* In later versions, the **corresponding tables will be directly exported from the database**.
* Merge by_id_ and discard records with duplicate titles and/or definitions.


In [2]:
dat1= pd.read_csv("ESTAT_dat_concepts_2021_04_08.csv",sep=";")
dat2= pd.read_csv("ESTAT_dat_link_info_2021_04_08.csv",sep=";")
Gloss_concepts = pd.merge(dat1,dat2,on=['id'])
del(dat1,dat2)

Gloss_concepts = Gloss_concepts[['id','title','definition']]

Gloss_concepts = Gloss_concepts.drop_duplicates(subset=["definition"])
Gloss_concepts = Gloss_concepts.dropna(axis=0,subset=["definition"])
Gloss_concepts = Gloss_concepts.drop_duplicates(subset=["title"])
Gloss_concepts = Gloss_concepts.dropna(axis=0,subset=["title"])

Gloss_concepts.reset_index(drop=True, inplace=True)
Gloss_concepts

,id,title,definition
0,1,Arrival of tourist at a tourist accommodation ...,See arrival disambiguation page fo...
1,3,Nights spent,A night spent or tourism night ...
2,4,Supply side tourism statistics,Regulation (EU) No 692/2011 of 6 July 2011 e...
3,5,Tourist,"For tourism statistics, a travell..."
4,7,Asia-Europe Meeting (ASEM),"The Asia-Europe Meeting , abbreviated..."
...,...,...,...
1279,2879,Statistical territory,The statistical territory of the ...
1280,2881,Statistical indicator,A statistical indicator is the repres...
1281,2885,Tourism industries,For the article Tourism industries - eco...
1282,2891,Sawnwood,Sawnwood is wood that has been produced ...


## Step 3. Pre-processing (cont)

* Keep a backup of the original definition.
* Do some data cleansing.


In [3]:
Gloss_concepts['definition_init'] = Gloss_concepts['definition']

Gloss_concepts['definition'] = Gloss_concepts['definition'].apply(lambda x: re.sub("[^a-zA-Z]", " ",x)) ## replace anything except digits and letters by space 
Gloss_concepts['definition'] = Gloss_concepts['definition'].apply(lambda x: re.sub(' +', ' ',x)) ## remove more than one spaces
Gloss_concepts['definition'] = Gloss_concepts['definition'].apply(lambda x: re.sub('^ +| +$', '',x)) ## remove start and end spaces
Gloss_concepts['definition'] = Gloss_concepts['definition'].apply(lambda x: re.sub(' , ',', ',x)) ## space-comma-space -> comma-space

Gloss_concepts


,id,title,definition,definition_init
0,1,Arrival of tourist at a tourist accommodation ...,See arrival disambiguation page for other mean...,See arrival disambiguation page fo...
1,3,Nights spent,A night spent or tourism night overnight stay ...,A night spent or tourism night ...
2,4,Supply side tourism statistics,Regulation EU No of July establishes a common ...,Regulation (EU) No 692/2011 of 6 July 2011 e...
3,5,Tourist,For tourism statistics a traveller is someone ...,"For tourism statistics, a travell..."
4,7,Asia-Europe Meeting (ASEM),The Asia Europe Meeting abbreviated as ASEM is...,"The Asia-Europe Meeting , abbreviated..."
...,...,...,...,...
1279,2879,Statistical territory,The statistical territory of the European Unio...,The statistical territory of the ...
1280,2881,Statistical indicator,A statistical indicator is the representation ...,A statistical indicator is the repres...
1281,2885,Tourism industries,For the article Tourism industries economic an...,For the article Tourism industries - eco...
1282,2891,Sawnwood,Sawnwood is wood that has been produced either...,Sawnwood is wood that has been produced ...


## Step 4. Apply the NER engine

* Create columns _ORG_, _GPE_, _NORP_, _LOCATION_ which will hold dictionaries with entities recognized as: 
    * Organizations ;  
    * Countries, cities, states ; 
    * Nationalities or religious or political groups ; 
    * Non-GPE locations, mountain ranges, bodies of water, respectively
* In each dictionary in a record, the key is the entity and  the value is the count in the definition of the article.


In [4]:
nlp.max_length = 1500000

Gloss_concepts['ORG'] = [dict() for i in range(len(Gloss_concepts))]
Gloss_concepts['GPE'] = [dict() for i in range(len(Gloss_concepts))]
Gloss_concepts['NORP'] = [dict() for i in range(len(Gloss_concepts))]
Gloss_concepts['LOCATION'] = [dict() for i in range(len(Gloss_concepts))]

for i in range(len(Gloss_concepts)):
    if i % 100 == 0: print('i = ',i,' of ',len(Gloss_concepts))
    tokens = nlp(Gloss_concepts.loc[i,'definition'])
    entities = tokens.ents
    for ent in entities:
        #print(ent.text, ent.label_)
        if ent.label_ == 'ORG':
            if ent.text.upper() in Gloss_concepts.loc[i,'ORG'].keys():
                Gloss_concepts.loc[i,'ORG'][ent.text.upper()] += 1
            else:    
                Gloss_concepts.loc[i,'ORG'][ent.text.upper()] = 1
        elif ent.label_ == 'GPE':
            if ent.text.upper() in Gloss_concepts.loc[i,'GPE'].keys():
                Gloss_concepts.loc[i,'GPE'][ent.text.upper()] += 1
            else:    
                Gloss_concepts.loc[i,'GPE'][ent.text.upper()] = 1
        elif ent.label_ == 'NORP':
            if ent.text.upper() in Gloss_concepts.loc[i,'NORP'].keys():
                Gloss_concepts.loc[i,'NORP'][ent.text.upper()] += 1
            else:    
                Gloss_concepts.loc[i,'NORP'][ent.text.upper()] = 1                
        elif ent.label_ == 'LOCATION':
            if ent.text.upper() in Gloss_concepts.loc[i,'LOCATION'].keys():
                Gloss_concepts.loc[i,'LOCATION'][ent.text.upper()] += 1
            else:    
                Gloss_concepts.loc[i,'LOCATION'][ent.text.upper()] = 1                
    
Gloss_concepts

#PERSON People, including fictional
#NORP Nationalities or religious or political groups
#FACILITY Buildings, airports, highways, bridges, etc.
#ORGANIZATION Companies, agencies, institutions, etc.
#GPE Countries, cities, states
#LOCATION Non-GPE locations, mountain ranges, bodies of water
#PRODUCT Vehicles, weapons, foods, etc. (Not services)
#EVENT Named hurricanes, battles, wars, sports events, etc.
#WORK OF ART Titles of books, songs, etc.
#LAW Named documents made into laws 
#LANGUAGE Any named language
#The following values are also annotated in a style similar to names:
#DATE Absolute or relative dates or periods
#TIME Times smaller than a day
#PERCENT Percentage (including “%”)
#MONEY Monetary values, including unit
#QUANTITY Measurements, as of weight or distance
#ORDINAL “first”, “second”
#CARDINAL Numerals that do not fall under another typ



i =  0  of  1284
i =  100  of  1284
i =  200  of  1284
i =  300  of  1284
i =  400  of  1284
i =  500  of  1284
i =  600  of  1284
i =  700  of  1284
i =  800  of  1284
i =  900  of  1284
i =  1000  of  1284
i =  1100  of  1284
i =  1200  of  1284


,id,title,definition,definition_init,ORG,GPE,NORP,LOCATION
0,1,Arrival of tourist at a tourist accommodation ...,See arrival disambiguation page for other mean...,See arrival disambiguation page fo...,{'EUROPEAN UNION EU': 1},{},{},{}
1,3,Nights spent,A night spent or tourism night overnight stay ...,A night spent or tourism night ...,{},{},{},{}
2,4,Supply side tourism statistics,Regulation EU No of July establishes a common ...,Regulation (EU) No 692/2011 of 6 July 2011 e...,{'ACCOMMODATION': 1},{},{'EUROPEAN': 1},{}
3,5,Tourist,For tourism statistics a traveller is someone ...,"For tourism statistics, a travell...",{},{},{},{}
4,7,Asia-Europe Meeting (ASEM),The Asia Europe Meeting abbreviated as ASEM is...,"The Asia-Europe Meeting , abbreviated...","{'ASEM': 2, 'THE EUROPEAN UNION EU': 1, 'EU': ...","{'OCEANIA': 1, 'NORWAY': 1, 'SWITZERLAND': 1, ...",{},{}
...,...,...,...,...,...,...,...,...
1279,2879,Statistical territory,The statistical territory of the European Unio...,The statistical territory of the ...,"{'THE EUROPEAN UNION EU': 1, 'EU': 1, 'MAYOTTE...","{'STATES': 1, 'GERMANY': 1, 'HELIGOLAND': 1, '...",{'FRENCH': 1},{}
1280,2881,Statistical indicator,A statistical indicator is the representation ...,A statistical indicator is the repres...,{'EUROPEAN UNION EU': 1},{},{},{}
1281,2885,Tourism industries,For the article Tourism industries economic an...,For the article Tourism industries - eco...,"{'UNWTO': 1, 'H INLAND': 1, 'HOTELS': 2, 'REST...",{},{'EUROPEAN': 1},{}
1282,2891,Sawnwood,Sawnwood is wood that has been produced either...,Sawnwood is wood that has been produced ...,{'SAWNWOOD': 1},{},{},{}


## Step 5. Gathering the most common entities: example with ORG entities 

We can see several errors and repetitions. These require some further cleansing steps and **fine-tuning of the NER engine** (not yet carried out).

In [5]:


from itertools import chain
org_list=sorted(list(chain.from_iterable(Gloss_concepts['ORG'].apply(lambda x: x.keys()))))
org_all_freqs = sorted(Counter(org_list))
print(org_all_freqs)



['A EUROPEAN UNION', 'A HOLDINGS', 'AAA', 'AAGR', 'ABBREVIATION DEGURBA SHORT', 'ABOVE NUTS', 'ACCOMMODATION', 'ACID', 'ACL', 'ACP', 'ADJUSTED', 'ADSL', 'AEA', 'AEI', 'AES', 'AF', 'AGENCY', 'AGRICULTURAL', 'AGRICULTURE', 'AIC', 'AIR', 'AL SERBIA RS', 'ALL EUROPEAN UNION EU', 'AMMONIA NH', 'AMOUNTS', 'ANALOGOUSLY', 'ANIMAL', 'ANSER', 'ANTICIPATED', 'ANTIGUA AND BARBUDA AG REPUBLIC OF GUINEA GN', 'APEC', 'APPLIED', 'ARA', 'ARAS BERRIES', 'ARCHITECTURAL', 'AREAS', 'ARMENIA AM BELARUS', 'AROPE', 'ART A OF', 'ARTICLE', 'ASEAN', 'ASEM', 'ASIA PACIFIC ECONOMIC COOPERATION', 'ASYLUM', 'ASYMMETRIC', 'AU', 'AU AUSTRALIA FJ', 'AUSTRALIA BANGLADESH', 'AUSTRALIA NOV', 'AUSTRIA AT ESTONIA EE ITALY', 'AUSTRIA BELGIUM', 'AUSTRIA FRANCE', 'AUTOPSY', 'AWU', 'B MINING', 'BANK', 'BASIC', 'BEEFALO BOS TAURUS', 'BELGIUM BE', 'BELGIUM BE GERMANY DE ESTONIA', 'BELGIUM BE LATVIA', 'BELOW', 'BERK PEGLER OYSTER', 'BGN', 'BIOMASS', 'BIRDS', 'BIS', 'BMI', 'BOARD OF APPEAL', 'BOD', 'BORDER', 'BOTSWANA', 'BPM', 'BRE

In [6]:
print('\n100 most common:\n')
org_common_freqs = Counter(org_list).most_common(100)
org_common = sorted([x[0] for x in org_common_freqs])
print(org_common)



100 most common:

['AEI', 'AGRICULTURAL', 'ALL EUROPEAN UNION EU', 'AREAS', 'BPM', 'CAP', 'CIS', 'CLASSIFICATION', 'COMMISSION', 'COUNCIL', 'COUNCIL REGULATION EC NO', 'CREDIT', 'CROPS', 'DATA', 'DIGITAL', 'DIRECTIVE EC', 'DMC', 'EC', 'EDP', 'EEA', 'EFTA', 'ESA', 'ESS', 'EU', 'EURO', 'EUROBASE', 'EUROPEAN BUSINESS STATISTICS REGULATION', 'EUROPEAN ENVIRONMENTAL ECONOMIC ACCOUNTS', 'EUROPEAN UNION', 'EUROPEAN UNION EU', 'EUROSTAT', 'EXCHANGE', 'EXTRA EU', 'FDI', 'FERTILISERS', 'FSS', 'GFCF', 'GNI', 'GOODS', 'HICP', 'HOUSEHOLDS', 'HRST', 'ICD', 'ICT', 'INCOME AND LIVING CONDITIONS', 'INTEGRATED FARM STATISTICS', 'INTERNATIONAL', 'INTRA EU', 'IPC', 'ISCO', 'KAU', 'LAU', 'LFS', 'LSU', 'LUCAS', 'METADATA EXCHANGE', 'MFA', 'NA', 'NACE', 'NACE REV', 'NATIONAL STATISTICAL INSTITUTES', 'NL', 'NPIS', 'NPISH', 'NUAA', 'NUTS', 'OECD', 'PPS', 'REGULATION EC NO', 'SAPM', 'SBS', 'SDR', 'SES', 'SNA', 'SO', 'STATE', 'STATISTICAL', 'STATISTICAL DATA', 'STS', 'TAXES', 'THE COUNCIL OF DECEMBER', 'THE COU

## Step 6. Storing information on these most common entities per article: example with ORG entities 

This is one way of storing the information on both all entities and counts and on the most common ones in a Pandas dataframe.

In [7]:

Gloss_concepts['ORG_COMMON_100'] = Gloss_concepts['ORG'].apply(lambda x: {y:x[y] for y in x.keys() if y in org_common})
Gloss_concepts

,id,title,definition,definition_init,ORG,GPE,NORP,LOCATION,ORG_COMMON_100
0,1,Arrival of tourist at a tourist accommodation ...,See arrival disambiguation page for other mean...,See arrival disambiguation page fo...,{'EUROPEAN UNION EU': 1},{},{},{},{'EUROPEAN UNION EU': 1}
1,3,Nights spent,A night spent or tourism night overnight stay ...,A night spent or tourism night ...,{},{},{},{},{}
2,4,Supply side tourism statistics,Regulation EU No of July establishes a common ...,Regulation (EU) No 692/2011 of 6 July 2011 e...,{'ACCOMMODATION': 1},{},{'EUROPEAN': 1},{},{}
3,5,Tourist,For tourism statistics a traveller is someone ...,"For tourism statistics, a travell...",{},{},{},{},{}
4,7,Asia-Europe Meeting (ASEM),The Asia Europe Meeting abbreviated as ASEM is...,"The Asia-Europe Meeting , abbreviated...","{'ASEM': 2, 'THE EUROPEAN UNION EU': 1, 'EU': ...","{'OCEANIA': 1, 'NORWAY': 1, 'SWITZERLAND': 1, ...",{},{},"{'THE EUROPEAN UNION EU': 1, 'EU': 1, 'EFTA': 1}"
...,...,...,...,...,...,...,...,...,...
1279,2879,Statistical territory,The statistical territory of the European Unio...,The statistical territory of the ...,"{'THE EUROPEAN UNION EU': 1, 'EU': 1, 'MAYOTTE...","{'STATES': 1, 'GERMANY': 1, 'HELIGOLAND': 1, '...",{'FRENCH': 1},{},"{'THE EUROPEAN UNION EU': 1, 'EU': 1}"
1280,2881,Statistical indicator,A statistical indicator is the representation ...,A statistical indicator is the repres...,{'EUROPEAN UNION EU': 1},{},{},{},{'EUROPEAN UNION EU': 1}
1281,2885,Tourism industries,For the article Tourism industries economic an...,For the article Tourism industries - eco...,"{'UNWTO': 1, 'H INLAND': 1, 'HOTELS': 2, 'REST...",{},{'EUROPEAN': 1},{},"{'SBS': 1, 'STS': 1}"
1282,2891,Sawnwood,Sawnwood is wood that has been produced either...,Sawnwood is wood that has been produced ...,{'SAWNWOOD': 1},{},{},{},{}


## Step 7. Exporting the dataframe to Excel 

This is useful for the manual inspection and the design of rules for the **fine-tuning of the NER engine**. This output can then directly be imported in the database.


In [8]:
Gloss_concepts.to_excel('Gloss_concepts.xlsx')